# Dataset from ProtonX School

## 1. Import the dataset and libraries

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt \
  -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

--2020-10-24 04:48:44--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-10-24 04:48:44 (66.8 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [ ]:
corpus = data.lower().split("\n")

In [ ]:
corpus

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 'when forty winters shall beseige thy brow,',
 "and dig deep trenches in thy beauty's field,",
 "thy youth's proud livery, so gazed on now,",
 "will be a tatter'd weed, of small worth held:",
 "then being ask'd where all thy beauty lies,",
 'where all the treasure of thy lusty days,',
 'to say, within thine own deep-sunken eyes,',

In [ ]:
# Import necessaries libraries
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

## 2. Preprocess the data

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus: 
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)): 
    n_gram_sequence = token_list[:i+1]
    print(n_gram_sequence)
    input_sequences.append(n_gram_sequence)

Streaming output truncated to the last 5000 lines.
[153, 5, 174, 3]
[153, 5, 174, 3, 856]
[153, 5, 174, 3, 856, 857]
[66, 209]
[66, 209, 2595]
[66, 209, 2595, 287]
[66, 209, 2595, 287, 150]
[66, 209, 2595, 287, 150, 2596]
[858, 206]
[858, 206, 1]
[858, 206, 1, 1299]
[858, 206, 1, 1299, 13]
[858, 206, 1, 1299, 13, 23]
[858, 206, 1, 1299, 13, 23, 5]
[858, 206, 1, 1299, 13, 23, 5, 400]
[858, 206]
[858, 206, 1]
[858, 206, 1, 1299]
[858, 206, 1, 1299, 2597]
[858, 206, 1, 1299, 2597, 3]
[858, 206, 1, 1299, 2597, 3, 160]
[858, 206, 1, 1299, 2597, 3, 160, 249]
[1, 7]
[1, 7, 29]
[1, 7, 29, 205]
[1, 7, 29, 205, 13]
[1, 7, 29, 205, 13, 5]
[1, 7, 29, 205, 13, 5, 488]
[1, 7, 29, 205, 13, 5, 488, 411]
[348, 2598]
[348, 2598, 7]
[348, 2598, 7, 66]
[348, 2598, 7, 66, 26]
[348, 2598, 7, 66, 26, 1298]
[348, 2598, 7, 66, 26, 1298, 471]
[348, 2598, 7, 66, 26, 1298, 471, 1258]
[858, 206]
[858, 206, 1]
[858, 206, 1, 75]
[858, 206, 1, 75, 147]
[858, 206, 1, 75, 147, 39]
[858, 206, 1, 75, 147, 39, 985]
[858, 

In [ ]:
import numpy as np
from keras.utils import to_categorical


max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, 
                                         padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = to_categorical(label, num_classes=total_words)

## 3. Create the model

In [ ]:
model = Sequential([
                    Embedding(total_words, 100, 
                              input_length=max_sequence_len-1), 
                    Bidirectional(LSTM(150, 
                                       return_sequences=True)), 
                    Dropout(0.2), 
                    LSTM(100), 
                    Dense(total_words/2, 
                          activation='relu', 
                          kernel_regularizer=regularizers.l2(0.01)), 
                    Dense(total_words, activation='softmax'), 
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

### Train the model

In [ ]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
484/484 [==============================] - 6s 12ms/step - loss: 6.9076 - acc: 0.0211
Epoch 2/100
484/484 [==============================] - 6s 12ms/step - loss: 6.5026 - acc: 0.0221
Epoch 3/100
484/484 [==============================] - 6s 13ms/step - loss: 6.3945 - acc: 0.0249
Epoch 4/100
484/484 [==============================] - 6s 13ms/step - loss: 6.2609 - acc: 0.0304
Epoch 5/100
484/484 [==============================] - 6s 12ms/step - loss: 6.1699 - acc: 0.0359
Epoch 6/100
484/484 [==============================] - 6s 13ms/step - loss: 6.0791 - acc: 0.0361
Epoch 7/100
484/484 [==============================] - 6s 13ms/step - loss: 5.9984 - acc: 0.0399
Epoch 8/100
484/484 [==============================] - 6s 13ms/step - loss: 5.9083 - acc: 0.0447
Epoch 9/100
484/484 [==============================] - 6s 13ms/step - loss: 5.8005 - acc: 0.0539
Epoch 10/100
484/484 [==============================] - 6s 13ms/step - loss: 5.6857 - acc: 0.0609
Epoch 11/100
484/484 [=======

## 4. Do the prediction

In [ ]:
test_seq = 'despite of wrinkles'

next_words = 10

for _ in range(next_words): 
  token_list = tokenizer.texts_to_sequences([test_seq])[0]
  token_list = pad_sequences([token_list], 
                             maxlen=max_sequence_len-1, 
                             padding='pre')
  
  predicted = model.predict_classes(token_list, verbose=0)
  output_word = ""

  for word, index in tokenizer.word_index.items(): 
    if index == predicted: 
      output_word = word
      break 
  
  test_seq += " " + output_word
  print(test_seq)

despite of wrinkles this
despite of wrinkles this thy
despite of wrinkles this thy golden
despite of wrinkles this thy golden time
despite of wrinkles this thy golden time of
despite of wrinkles this thy golden time of youth
despite of wrinkles this thy golden time of youth so
despite of wrinkles this thy golden time of youth so thee
despite of wrinkles this thy golden time of youth so thee bearing
despite of wrinkles this thy golden time of youth so thee bearing thee
